In [1]:
#import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pickle
import yaml

import importlib

import Model10_OOP_small
importlib.reload(Model10_OOP_small)
from Model10_OOP_small import ConvModel as ConvModel_small

import BaseData20Fold
importlib.reload(BaseData20Fold)
from BaseData20Fold import DataProcessor

import Plots
importlib.reload(Plots)
from Plots import plot_CM


# COntains 2stream Neural Network to process fine fatures and coarse features separate


In [2]:

cwd = Path.cwd()
%load_ext tensorboard
%load_ext version_information
#!rm -rf ./logs/

logsdir = cwd / '..' / 'logs'
# !rmdir /s /q {logsdir}
#!tensorboard --logdir {logsdir} --host localhost --port 6006

cfg = 'config_mdl10_small.yaml'
with open(cfg, 'r') as file:
    config = yaml.safe_load(file)
seeds = config['seeds']

folds = np.arange(1,21)

datapath = cwd / '..' / 'Data' / 'DHG2016' / 'small_windows'

In [ ]:
resultsDFlong = pd.DataFrame()
folder = 'small_windows'
data_processor = DataProcessor(cfg)
data_processor.load_handgestdata()
data_processor.handangles2windows()
data_processor.save_config(folder)
data_processor.save_windowsets(folder)

#folds = [5]
    
for fold in folds:
    data_processor = DataProcessor(cfg, fold = fold)
    # data_processor.load_handgestdata()
    # data_processor.handangles2windows()
    # data_processor.save_windowsets(folder)
    data_processor.load_windows(folder)
    data_processor.processwindows()
    data_processor.save_windowsets_processed(folder, name='Fold'+str(fold))


In [12]:
datapath = cwd / '..' / 'Data' / 'DHG2016' / 'small_windows'

results_valid_labels_list = []
results_scores_list_valid = []
results_scores_list_train = []


for seedidx, n_fold in enumerate(folds):
    print('evaluating Fold', n_fold)
    seed_val = seeds[seedidx]

    fold = pickle.load(open((datapath / ('Fold' + str(n_fold) + '.pkl')).resolve(), 'rb'))
    Y_train_oh = fold['train']['Y_oh']
    Y_valid_oh = fold['valid']['Y_oh']
    X_train = fold['train']['X']
    X_valid = fold['valid']['X']
  

    like7_small = ConvModel_small(cfg, 'Test', seed = seed_val)
    like7_small.compile_model()

    # Train the model
    like7_small.train_model(X_train, Y_train_oh, X_valid, Y_valid_oh)

    Y_prob_train = like7_small.model.predict(X_train)
    Y_prob_valid = like7_small.model.predict(X_valid)
    Y_score_true = np.max(Y_prob_valid, axis=-1)

    Y_hat_train = np.argmax(Y_prob_train, axis=-1)
    Y_hat_valid = np.argmax(Y_prob_valid, axis=-1)

    results_train = like7_small.model.evaluate(X_train, Y_train_oh, verbose = 0, return_dict=True)
    results_valid = like7_small.model.evaluate(X_valid, Y_valid_oh, verbose = 0, return_dict=True)

    print(f'NLL: {results_valid['NLL']:,.2f}')
    #print(f'Validation score: {results['loss']:,.2f}')
    print(f'Validation accuracy: {results_valid['accuracy']:,.2f}')
    print(f'Validation F1Score: {results_valid['f1_score']:,.2f}')

    results_valid_labels_list.append(pd.DataFrame({'Fold': n_fold, 'Y': fold['valid']['Y'], 'Y_hat': Y_hat_valid, 'Y_prob': Y_score_true}))
    results_scores_list_train.append(pd.DataFrame(results_train, index = [n_fold]))
    results_scores_list_valid.append(pd.DataFrame(results_valid, index = [n_fold]))
    #results_scores_list.append({'Fold': n_fold, 'Accuracy_train': accuracy_train, 'Accuracy_valid': accuracy_valid, 'F1-Score_train': f1_score_train, 'F1-Score_valid': f1_score_valid, 'NLL_train': nll_train, 'NLL_valid': nll_valid})
    
results_scores = pd.concat([pd.concat(results_scores_list_train).add_suffix('_train'),pd.concat(results_scores_list_valid).add_suffix('_valid')],axis = 1)
results_scores.rename(columns = {'fold_train': 'fold'}, inplace = True)
results_valid_labels = pd.concat(results_valid_labels_list)


evaluating Fold 1
Setting seed to 5
Epoch 1/20
844/844 - 20s - 24ms/step - NLL: 1.5575 - accuracy: 0.5169 - f1_score: 0.2869 - loss: 1.8086 - precision: 0.6743 - recall: 0.3065 - val_NLL: 1.5190 - val_accuracy: 0.4870 - val_f1_score: 0.2990 - val_loss: 1.7153 - val_precision: 0.6432 - val_recall: 0.3608
Epoch 2/20
844/844 - 17s - 20ms/step - NLL: 1.2151 - accuracy: 0.6010 - f1_score: 0.4391 - loss: 1.4115 - precision: 0.7400 - recall: 0.4422 - val_NLL: 1.4016 - val_accuracy: 0.5265 - val_f1_score: 0.3692 - val_loss: 1.5854 - val_precision: 0.6599 - val_recall: 0.4246
Epoch 3/20
844/844 - 15s - 18ms/step - NLL: 1.1176 - accuracy: 0.6304 - f1_score: 0.4895 - loss: 1.3104 - precision: 0.7600 - recall: 0.4856 - val_NLL: 1.3411 - val_accuracy: 0.5438 - val_f1_score: 0.4072 - val_loss: 1.5235 - val_precision: 0.6653 - val_recall: 0.4432
Epoch 4/20
844/844 - 14s - 17ms/step - NLL: 1.0543 - accuracy: 0.6495 - f1_score: 0.5173 - loss: 1.2485 - precision: 0.7713 - recall: 0.5143 - val_NLL: 1.311

In [13]:
results_valid_labels.to_pickle(datapath/'results_valid_labels_dcnn.pkl')
results_scores.to_pickle(datapath/'results_scores_dcnn.pkl')

In [8]:
results_scores.columns

Index(['NLL_train', 'accuracy_train', 'f1_score_train', 'loss_train',
       'precision_train', 'recall_train', 'NLL_valid', 'accuracy_valid',
       'f1_score_valid', 'loss_valid', 'precision_valid', 'recall_valid'],
      dtype='object')

In [14]:
mean = results_scores.mean()
std = results_scores.std()

print(f'accuracy on training data - frames: {mean["accuracy_train"]*100:.1f}% +-  {std["accuracy_train"]*100:.1f}%')
print(f'F1_Score on training data - frames: {mean["f1_score_train"]*100:.1f}% +- {std["f1_score_train"]*100:.1f}%')
print(f'NLL on training data - frames: {mean["NLL_train"]:.2f} +- {std["NLL_train"]:.2f}')

print(f'accuracy on validation data - frames: {mean['accuracy_valid']*100:.1f}% +- {std['accuracy_valid']*100:.1f}%')
print(f'F1-Score on validation data - frames: {mean['f1_score_valid']*100:.1f}% +- {std['f1_score_valid']*100:.1f}%')
print(f'NLL on validation data - frames: {mean['NLL_valid']:.2f} +- {std['NLL_valid']:.2f}')

accuracy on training data - frames: 72.3% +-  1.4%
F1_Score on training data - frames: 63.0% +- 2.5%
NLL on training data - frames: 0.79 +- 0.04
accuracy on validation data - frames: 63.7% +- 4.2%
F1-Score on validation data - frames: 49.0% +- 7.0%
NLL on validation data - frames: 1.13 +- 0.16


In [11]:
mean = results_scores.mean()
std = results_scores.std()

print(f'accuracy on training data - frames: {mean["accuracy_train"]*100:.1f}% +-  {std["accuracy_train"]*100:.1f}%')
print(f'F1_Score on training data - frames: {mean["f1_score_train"]*100:.1f}% +- {std["f1_score_train"]*100:.1f}%')
print(f'NLL on training data - frames: {mean["NLL_train"]:.2f} +- {std["NLL_train"]:.2f}')

print(f'accuracy on validation data - frames: {mean['accuracy_valid']*100:.1f}% +- {std['accuracy_valid']*100:.1f}%')
print(f'F1-Score on validation data - frames: {mean['f1_score_valid']*100:.1f}% +- {std['f1_score_valid']*100:.1f}%')
print(f'NLL on validation data - frames: {mean['NLL_valid']:.2f} +- {std['NLL_valid']:.2f}')


accuracy on training data - frames: 58.3% +-  2.2%
F1_Score on training data - frames: 60.0% +- 2.2%
NLL on training data - frames: 1.16 +- 0.06
accuracy on validation data - frames: 47.0% +- 5.2%
F1-Score on validation data - frames: 45.1% +- 5.9%
NLL on validation data - frames: 1.56 +- 0.17


In [ ]:
%version_information tensorflow, numpy, matplotlib, plotly, pandas, keras